In [ ]:
import os
import tarfile
import urllib
import sys

def download_and_cache(url, fname=None, dest='/tmp/tensorflow_tutorials'):
    if not os.path.exists(dest):
        os.makedirs(dest)
    if fname is None:
        fname = url.split('/')[-1]
    fpath = os.path.join(dest, fname)
    if not os.path.exists(fpath):
        def _progress(count, block_size, total_size):
            percentage = float(count * block_size) / float(total_size) * 100.0
            sys.stdout.write('\r>> Downloading {} {:1.1f}%'.format(fname, percentage))
            sys.stdout.flush()
        fpath, _ = urllib.request.urlretrieve(url, fpath, _progress)
        print()
        statinfo = os.stat(fpath)
        print('Successfully downloaded', fname, statinfo.st_size, 'bytes.')
    return fpath
    

In [ ]:
fpath = download_and_cache('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')

In [ ]:
fpath

In [ ]:
tar = tarfile.open(fpath, 'r:gz')

In [ ]:
for c, member in enumerate(tar.getmembers()):
    f=tar.extractfile(member)
    print(member.name, member.type)
    if c > 20:
        break